In [1]:
#!pip install tensorboard tensorflow pandas

"""
If tensorboard is not installed (or other dependencies, such as tensorflow and pandas),
uncomment the command in top and re-run. This needs only to be run once in a Jupyter kernel.
"""

%load_ext tensorboard

from tensorflow.python.summary.summary_iterator import summary_iterator
import pandas as pd

2022-11-01 14:18:54.836001: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
"""
Change the LOG_DIR argument to point to the correct directory, you may want to use an
absolute path if you run into issues.
"""
%tensorboard --logdir data/64/logs-64-30-2

In [2]:
def logs_to_pandas(path: str) -> pd.DataFrame:
    """convert single tensorflow log file to pandas DataFrame
    Parameters
    ----------
    path : str
        path to tensorflow log file
    Returns
    -------
    pd.DataFrame
        converted dataframe
    """

    runlog_data = pd.DataFrame({"metric": [], "value": [], "step": [], "wall_time": []})
    try:
        event_acc = summary_iterator(path)
        for event in list(event_acc)[1:]:
            step, wall_time = event.step, pd.to_datetime(event.wall_time, unit='s')
            simple_extractor = [{"metric": v.tag, "value": v.simple_value, "step": step, 'wall_time': wall_time} for v in event.summary.value]
            event_r = pd.DataFrame(simple_extractor)
            runlog_data = pd.concat([runlog_data, event_r])
    # Dirty catch of DataLossError
    except Exception as e:
        print("Event file possibly corrupt: {}".format(path))
        print(e)
    return runlog_data

In [3]:
from glob import glob
import os
import numpy as np

In [7]:
for lambda_ in [15, 20]:
    with open(f'service_time_{lambda_}.txt', 'w') as f:
        for r_type in glob(f'data/l{lambda_}-128-35-3'):
            for log_path in glob(os.path.join(r_type,'*.log')):
                arrival_times = []
                server_log_data  = open(log_path)
                service_avg = np.array([])
                for l in server_log_data.readlines():
                    if l.find('Scheduling') >=0:
                        node_arrival_times = ' '.join(l.split(' ')[0:2])
                        label = l.strip().split(' ')[-1]
                        files = glob(os.path.join(r_type, '0/cloud_experiment_'+label+'/*/*/*'))
                        for file in files:
                            log_df = logs_to_pandas(file)
                            acc_df = log_df.loc[log_df['metric'] == 'accuracy per epoch']
                            service_start_time = acc_df.iloc[0]['wall_time']
                            end_times = acc_df.iloc[-1]['wall_time']
                            training_time = end_times - service_start_time
                            service_avg = np.append(service_avg, training_time)

                print('configuration: ',log_path.split('/')[-1].split('.')[0], file=f)
                print(service_avg, file=f)
                print('mean service time = ', np.mean(service_avg),file=f)
        f.close()

In [8]:
# for batch_size in [64, 128, 256]:
for lambda_ in [15, 20]:
    with open(f'response_time_{lambda_}.txt', 'w') as f:
        for r_type in glob(f'data/l{lambda_}-128-35-3'):
            for log_path in glob(os.path.join(r_type,'*.log')):
                server_log_data  = open(log_path)
                service_avg = np.array([])
                response_avg = np.array([])
                for l in server_log_data.readlines():
                    if l.find('Scheduling') >=0:
                        node_arrival_times = pd.Timestamp(' '.join(l.split(' ')[0:2]))
                        label = l.strip().split(' ')[-1]
                        files = glob(os.path.join(r_type, '0/cloud_experiment_'+label+'/*/*/*'))
                        for file in files:
                            log_df = logs_to_pandas(file)
                            acc_df = log_df.loc[log_df['metric'] == 'accuracy per epoch']
                            service_start_time = acc_df.iloc[0]['wall_time']
                            end_times = acc_df.iloc[-1]['wall_time']
                            service_time = end_times - service_start_time
                            response_time = end_times - node_arrival_times
                            response_avg = np.append(response_avg, response_time)
                            service_avg = np.append(service_avg, service_time)
                
                print('configuration: ',log_path.split('/')[-1].split('.')[0], file=f)
                print(response_avg, file=f)
                print('mean response time = ', np.mean(response_avg),file=f)
        f.close()

In [6]:
for batch_size in [64, 128, 256]:
    with open(f'accuracy_{batch_size}.txt', 'w') as f:
        for r_type in glob(f'data/{batch_size}/*'):
            for log_path in glob(os.path.join(r_type,'*.log')):
                server_log_data  = open(log_path)
                acc_avg = np.array([])
                for l in server_log_data.readlines():
                    if l.find('Scheduling') >=0:
                        label = l.strip().split(' ')[-1]
                        files = glob(os.path.join(r_type, '0/cloud_experiment_'+label+'/*/*/*'))
                        for file in files:
                            log_df = logs_to_pandas(file)
                            acc_df = log_df.loc[log_df['metric'] == 'accuracy per epoch']
                            acc = acc_df.iloc[-1]['value']
                            acc_avg = np.append(acc_avg, acc)
                
                print('configuration: ',log_path.split('/')[-1].split('.')[0], file=f)
                print(acc_avg, file=f)
                print('mean accuracy = ', np.mean(acc_avg),file=f)
        f.close()

In [117]:
acc_df

,metric,value,step,wall_time
0,accuracy per epoch,81.599998,1.0,2022-10-30 14:56:20.588840960
0,accuracy per epoch,83.760002,2.0,2022-10-30 14:57:34.897481728
0,accuracy per epoch,85.120003,3.0,2022-10-30 14:58:47.295026432
0,accuracy per epoch,86.480003,4.0,2022-10-30 14:59:59.007947264
0,accuracy per epoch,87.720001,5.0,2022-10-30 15:01:11.896925184
0,accuracy per epoch,88.199997,6.0,2022-10-30 15:02:25.590745600
0,accuracy per epoch,88.400002,7.0,2022-10-30 15:03:38.496906496
0,accuracy per epoch,88.559998,8.0,2022-10-30 15:04:51.787293952
0,accuracy per epoch,88.279999,9.0,2022-10-30 15:06:04.789909504
0,accuracy per epoch,87.559998,10.0,2022-10-30 15:07:17.194273792


In [115]:
acc_df.iloc[-1]['wall_time'] - pd.Timestamp("10-27-2022 18:01:54")

Timedelta('2 days 21:28:27.983843584')

In [32]:
log_df = logs_to_pandas(files[0])

In [73]:
acc_df = log_df.loc[log_df['metric'] == 'accuracy per epoch']

In [84]:
service_time = acc_df.iloc[-1]['wall_time'] - acc_df.iloc[0]['wall_time']

In [86]:
type(service_time)

pandas._libs.tslibs.timedeltas.Timedelta

In [104]:
log_df[log_df['step']==1]['wall_time'].iloc[0]

Timestamp('2022-10-30 14:56:20.588718080')